In [1]:
# pip install tiktoken -q

In [2]:
# pip install pandas

In [3]:
! python --version

Python 3.11.7


In [4]:
# !pip install langchain-community -q

In [5]:
# pip install langchain -q

In [3]:
import time
import pandas as pd
import json
import os
from langchain_community.chat_models import ChatOllama
from langchain_community.llms import Ollama
from langchain.chains import LLMChain
from langchain.prompts.chat import (ChatPromptTemplate, HumanMessagePromptTemplate, SystemMessagePromptTemplate)
from langchain import PromptTemplate
from langchain_core.prompts import FewShotPromptTemplate, PromptTemplate
from langchain.schema import (SystemMessage, HumanMessage, AIMessage)
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain.docstore.document import Document
import re

In [5]:
#examples
# df=pd.read_csv("Bio_with_only_reports_csv_1.csv")
df=pd.read_excel("ind_sample_demo.xlsx")
# texts=df["V_BIOGRAPHY_repo"].to_list()

In [83]:
patt=re.compile(r"(\[Reports\].*)",re.IGNORECASE)
df["V_BIOGRAPHY_repo"]=df["V_BIOGRAPHY_x"].str.extract(patt)

In [84]:
df.shape,df[df["V_BIOGRAPHY_x"].str.contains(patt)].shape

C:\Users\admin\AppData\Local\Temp\ipykernel_15720\2446129400.py:1: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  df.shape,df[df["V_BIOGRAPHY_x"].str.contains(patt)].shape


((39, 118), (39, 118))

In [85]:
df["V_BIOGRAPHY_repo"]=df["V_BIOGRAPHY_repo"].str.replace("[REPORTS]","")

In [86]:
# df["V_BIOGRAPHY_repo"]=df["V_BIOGRAPHY_repo"].str.lower()

In [88]:
patt=re.compile("(\s+|)To be determined(\.|\s*)",re.IGNORECASE)
df["V_BIOGRAPHY_repo"]=df["V_BIOGRAPHY_repo"].apply(lambda x: re.sub(patt,"",x))

## Prompt 6i

In [150]:
import random
random.seed(8)
examples_6i = [ {"input":" may 2012 - sentenced by xihu district court to 3 years imprisonment for extortion (cny6.57m) (sep 2011).",
                "output":[{"head":"May 2012","relation":"Judgement","tail":"sentenced by xihu district court to 3 years imprisonment","tail_type":"Decision"\
                          },{"head":"May 2012","relation":"Convicted","tail":"extortion","tail_type":"Crime"},
                          {"head":"September 2011","head_type":"Date","relation":"CrimeCommitted","tail":"extortion","tail_type":"Crime"}]},
#19
            {"input":" jun 2022 - pleaded guilty at the magistrate court to the criminal charges of instigating employees in practicing dentistry without \
          authorisation from the brunei medical board (2019 - 2020). fined bnd24,000, in default, 24 months imprisonment.","output": [
        {
            "head": "June 2022",
            "head_type": "Date",
            "relation": "ChargedWithCrime",
            "tail": "instigating employees in practicing dentistry without authorisation from the brunei medical board",
            "tail_type": "Crime"
        },
        {
            "head": "2019 - 2020",
            "head_type": "Date",
            "relation": "CrimeCommited",
            "tail": "instigating employees in practicing dentistry without authorisation from the brunei medical board ",
            "tail_type": "Crime"
        }
    ,{   "head": "June 2022",
            "head_type": "Date",
            "relation": "Judgement",
            "tail": "fined bnd24,000",
            "tail_type": "Decision"}]
},
    {"input":" feb 2023 - reportedly fined bdt0.1m and sentenced to 10 years of \
    imprisonment for trafficking narcotics and seized (0.5m yaba tablets).",
    "output": [
        {
            "head": "February 2023",
            "head_type": "Date",
            "relation": "Judgement",
            "tail": "fined bdt0.1m and sentenced to 10 years of imprisonment for trafficking narcotics ",
            "tail_type": "Decision"
        },{"head": "February 2023",
            "head_type": "Date",
            "relation": "Convicted",
            "tail": "trafficking narcotics",
            "tail_type": "Crime"}
    ]
}, {
        "input": 'Oct 2022 - sentenced by Sampit District Court to 5 years \
            imprisonment and fined IDR1b in lieu of additional 3 months imprisonment for methamphetamine trafficking (Feb - 2021). \
            and last year he got an award as the Best Talent'
        ,
        "output":[{"head": "October 2022",
        "head_type": "Date",
        "relation": "Judgement",
        "tail": "sentenced by Sampit District Court to 5 years imprisonment and fined IDR1b in lieu of additional 3 months imprisonment ",
        "tail_type": "Decision"},
        {"head":"February 2021",
         "head_type":"Date",
         "relation":"CrimeCommited",
         "tail":"methamphetamine trafficking",
         "tail_type":"Crime"}
    ]},
              {
        "input": 'dec 2019 - pleaded not guilty to the criminal charges of bribery (myr295,000) (jul 2019 - oct 2019). bail allowed. \
        case mention scheduled for jan 2020. aug 2021 - ordered to enter defence. case mention scheduled for oct 2021. mar 2022 - \
        convicted by the sessions court on criminal charges of bribery. sentenced to 58 years imprisonment and \
        fined myr1.05m in default 72 months imprisonment. allowed stay of execution of the sentence pending appeal to the high court.'\
             , "output":[
        {"head": "July 2019-October 2019",
        "head_type": "Date",
        "relation": "CrimeCommitted",
        "tail": "Bribery",
        "tail_type": "Crime"},{ 
            "head":"Decemeber 2019",
             "head_type":"Date",
             "relation" :"ChargedwithCrime",
             "tail":"Bribery",
             "tail_type":"Crime"},
            {"head":"March 2022",
             "head_type":"Date",
             "relation" :"Convicted",
             "tail":"Bribery",
             "tail_type":"Crime"}
            
    ,{       "head":"March 2022",
             "head_type":"Date",
             "relation" :"Judgement",
             "tail":"sentenced to 58 years imprisonment and \
        fined myr1.05m in default 72 months imprisonment. allowed stay of execution of the sentence pending appeal to the high court.",
             "tail_type":"Decision"
        
    }]},
    
    #year
        {"input":"nov 2021 - expelled from the cpc and dismissed from the\
                    official position for bribery (cny68,400) (2017-2020). case transferred for prosecution.",
         "output": [
           { "head": "2017-2020",
            "head_type": "Date",
            "relation": "CrimeCommitted",
            "tail": "bribery",
            "tail_type": "Crime"},
             {
            "head": "November 2021",
            "head_type": "Date",
            "relation": "Judgement",
            "tail": "dismissed from the official position for bribery (cny68,400)",
            "tail_type": "Decision"
        },
        {
            "head": "November 2021",
            "head_type": "Date",
            "relation": "ChargedWithCrime",
            "tail": "bribery",
            "tail_type": "Crime"
        }

    ]},#year and sentence
     

         

    # year
     {
     "input":' may 2016 - fined rub30,000 by murom city court for organisation of illegal migration (nov 2015 - dec 2015).',
      "output":[{"head":"May 2016","head_type":"Date","relation":"Judgement","tail":"fined rub30,000","tial_type":"Decision"},{"head":"November 2015 - December 2015","head_type":"Date","relation":"CrimeCommitted","tail":"illegal migration","tial_type":"Crime"}]
    
     } 
# year

  # year and sentence 
  
# sentence

        
    ]



In [151]:
system_prompt ='''You are a top-tier algorithm designed for extracting information in structured formats to build a knowledge graph.
Your task is to identify the entities and relations from the input text. You must generate the output as a list containing
JSON objects. Each JSON object should be in format {"head":entity,"head_type":type of entity assigned to head, "tail":entity, "tail_type":type of entity assigned to tail,"relation":relation between entities asigned to head and tail}.
'''

In [152]:
class ExtractedInfo(BaseModel):
    head: str = Field(description="extracted first or head entity like Decmeber-2022 etc.")
    head_type: str = Field(description="type of the extracted head entity like Date etc.")
    relation: str = Field(description="relation between the head and the tail entities or if tail is none then relation be also none.")
    tail: str = Field(description="extracted second or tail entity like Bribery,Fined,Imprisioned, None etc.")
    tail_type: str = Field(description="type of the extracted tail entity like Crime, Sentence, None etc.")
    
parser = JsonOutputParser(pydantic_object=ExtractedInfo)

In [153]:
# all_mrgd_ot_gpu
llm=ChatOllama(model="llama3",temperature=0,format="json")
human_prompt=PromptTemplate(template="""
Below are a list of examples,each having a JSON object with input and output as keys.Input key takes {input}
and output key generates is list of JSON having extracted entity and relations.Generate output as per this format.

examples: {examples}
Use the following relation types, don't use other relation that is not defined below:
# RELATION TYPES:
{rel_types}

For the following input, extract entities and relationships as described in examples.
{format_instructions}
Input text: {input}
In the {input}, identify all date event relationships.

"""
,input_variables=["input"],partial_variables={"examples":examples_6i,"rel_types":["Convicted","CrimeCommited","Judgement","ChargedWithCrime"],
        "format_instructions": parser.get_format_instructions()})


human_message_prompt=HumanMessagePromptTemplate(prompt=human_prompt)


# "Date" and "Crime"  with relation as "Convicted".Don't provide any other entity and relationship

In [154]:
system_message=SystemMessage(content=system_prompt)
chat_prompt=ChatPromptTemplate.from_messages([system_message,human_message_prompt])

In [155]:


# prompt = ChatPromptTemplate.from_messages([system_message_prompt, human_message_prompt])
llm = ChatOllama(model = "llama3", temperature=0,format="json")
chain = LLMChain(llm=llm, prompt=chat_prompt)
# chain = chat_prompt | llm

In [156]:
texts=df.loc[:,"V_BIOGRAPHY_repo"].to_list()

In [157]:

llm = ChatOllama(model = "llama3", temperature=0,format="json")
chain = LLMChain(llm=llm, prompt=chat_prompt)
l6=[]
bio6=[]
ind6=[]
index6=0
time6=[]
for i in texts:
    print(index6)
    if i=="":
        l6.append({})
        time6.append(0)
        bio6.append(i)
        ind6.append(index6)
        index6=index6+1
    else:
       start6=time.time()
       output=chain.run(i)
       end6=time.time()
       time6.append(end6-start6)
       l6.append(output)
       bio6.append(i)
       ind6.append(index6)
       index6=index6+1

        

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38


In [158]:
df_res=pd.DataFrame({})
df_res["Biography"]=bio6
df_res["groups"]=l6
df_res["index"]=ind6


In [159]:
df_res["time"]=time6

In [160]:
df_res

,Biography,groups,index,time
0,"Nov 2017 - detained by Pakistan Rangers Sindh in Mominabad for involvement in several cases of terrorism and attack on polio workers. Apr 2022 - no further information reported.[NONCONVICTION TERROR CATEGORY NOTICE] This category includes information about individuals who are reportedly being investigated for, or have been arrested or charged on suspicion of, involvement in terrorism or terror related activities. The category also includes individuals or entities identified on national or internationally recognised banning, warning or wanted lists as allegedly connected to terrorism or individuals who are reportedly connected to an organisation included on any national or international terrorism list. Inclusion in this category does not mean that an individual or entity is a terrorist or terrorist organisation or that they are involved in or connected to terrorism or terror related activity. Individuals included in this category have not been convicted of any terror related activity; you should review the content carefully and in accordance with our terms and conditions, further enquiries should be made of the report subject to investigate the outcome of any alleged investigation, arrest, charges or any reported connection to any terror related activity and whether such allegations are denied.","{\n""head"": ""Nov 2017"", \n""head_type"": ""Date"", \n""relation"": ""CrimeCommitted"", \n""tail"": ""terrorism and attack on polio workers"", \n""tail_type"": ""Crime""\n}\n\n",0,15.098829
1,,{},1,0.000000
2,"Dec 2018 - reportedly detained. Sentenced to death for killing law enforcement agents and destruction of government educational institutions. Apr 2022 - no further information reported.[CRIME - TERROR CATEGORY NOTICE] This category includes information about individuals who have reportedly been convicted of a terror related crime. Inclusion in the category does not mean that an individual or entity is a terrorist or terrorist organisation. Individuals may appeal their conviction; you should review the content carefully and in accordance with our terms and conditions, further enquiries should be made of the report subject to investigate the conviction or the outcome of any appeal and whether the allegations of wrongdoing are denied.","{""head"": ""December 2018"", ""head_type"": ""Date"", ""relation"": ""Detained"", ""tail"": ""reportedly detained"", ""tail_type"": ""Event""}\n\n\n",2,5.301877
3,"Dec 2018 - reportedly detained. Sentenced to death for killing law enforcement agents and destruction of government educational institutions. Apr 2022 - no further information reported.[CRIME - TERROR CATEGORY NOTICE] This category includes information about individuals who have reportedly been convicted of a terror related crime. Inclusion in the category does not mean that an individual or entity is a terrorist or terrorist organisation. Individuals may appeal their conviction; you should review the content carefully and in accordance with our terms and conditions, further enquiries should be made of the report subject to investigate the conviction or the outcome of any appeal and whether the allegations of wrongdoing are denied.","{""head"": ""December 2018"", ""head_type"": ""Date"", ""relation"": ""Detained"", ""tail"": ""reportedly detained"", ""tail_type"": ""Event""}\n\n\n \n\n\n\n\n\n",3,4.187027
4,"Nov 2021 - reportedly defendant on a trial before the Third Circuit Court for Terrorism. Suspected of establishing a cell affiliated with ISIS, travelled abroad for weapons training (ISIS Helwan Cell Case). Nov 2021 - trail postponed for Dec. Jan 2022 - reportedly sentenced (unspecified term of sentencing).[CRIME - TERROR CATEGORY NOTICE] This category includes information about individuals who have reportedly been convicted of a terror related crime. Inclusion in the category does not mean that an individual or entity is a terrorist or terrorist organisation. Individuals may appeal their conviction; 

## Date of crime prompt

In [161]:
random.seed(8)
doc_2=[{"input":" apr 2022 - pleaded guilty at the sessions court to the criminal charges \
of bribery (myr20,000) (jun 2021). fined myr5,000 in default 1 month imprisonment.",
        "output":[{"head":"June 2021","head_type":"Date","relation":"CrimeCommitted","tail":"bribery","tail_type":"Crime"}]}

       
    ,      {"input":" feb 2023 - reportedly fined bdt0.1m and sentenced to 10 years of \
    imprisonment for trafficking narcotics and seized (0.5m yaba tablets).",
    "output": [{"head":None,"head_type":None,"relation":None,"tail":None,"tail_type":None}]},
 
 {"input":' 2010 - reportedly declared as proclaimed offender by khairpur court for alleged involvement in kidnapping. aug 2022 - no further information reported.',
            "output": [{"head":None,"head_type":None,"relation":None,"tail":None,"tail_type":None}]},
 {"input":" jun 2022 - pleaded guilty at the magistrate court to the criminal charges of instigating employees in practicing dentistry without \
          authorisation from the brunei medical board (2019 - 2020). fined bnd24,000, in default, 24 months imprisonment.",
  "output": [{
            "head": "2019 - 2020",
            "head_type": "Date",
            "relation": "CrimeCommited",
            "tail": "instigating employees in practicing dentistry without authorisation from the brunei medical board ",
            "tail_type": "Crime"
        }]},
     {"input":" may 2012 - sentenced by xihu district court to 3 years imprisonment for extortion (cny6.57m) (sep 2011).",
                "output":[{"head":"September 2011","head_type":"Date","relation":"CrimeCommitted","tail":"extortion","tail_type":"Crime"}]},


  {  "input": 'Oct 2022 - sentenced by Sampit District Court to 5 years \
            imprisonment and fined IDR1b in lieu of additional 3 months imprisonment for methamphetamine trafficking (Feb - 2021). \
            and last year he got an award as the Best Talent',
  "output":[{"head":"February 2021",
         "head_type":"Date",
         "relation":"CrimeCommited",
         "tail":"methamphetamine trafficking",
         "tail_type":"Crime"}]},{"input": ' 2009 - reportedly declared as proclaimed offender by ghotki court for alleged involvement in dacoity. jun 2022 - no further information reported.'
,"output": [{"head":None,"head_type":None,"relation":None,"tail":None,"tail_type":None}]}]

 


 
 

In [162]:
system_prompt ='''You are a top-tier algorithm designed for extracting information in structured formats to build a knowledge graph.
Your task is to identify the entities and relations from the input text. You must generate the output as a list containing
JSON objects. Each JSON object should be in format {"head":entity,"head_type":type of entity assigned to head, "tail":entity, "tail_type":type of entity assigned to tail,"relation":relation between entities asigned to head and tail}.
'''

In [163]:
class ExtractedInfo(BaseModel):
    head: str = Field(description="extracted first or head entity like Decmeber-2022 etc.")
    head_type: str = Field(description="type of the extracted head entity like Date etc.")
    relation: str = Field(description="relation between the head and the tail entities or if tail is none then relation be also none.")
    tail: str = Field(description="extracted second or tail entity like Bribery,Fined,Imprisioned, None etc.")
    tail_type: str = Field(description="type of the extracted tail entity like Crime, Sentence, None etc.")
    
parser = JsonOutputParser(pydantic_object=ExtractedInfo)

In [164]:
# all_mrgd_ot_gpu
llm=ChatOllama(model="llama3",temperature=0,format="json")
human_prompt=PromptTemplate(template="""
Below are a list of examples,each having a JSON object with input and output as keys.Input key takes {input}
and output key generates is list of JSON having extracted entity and relations.Generate output as per this format.

examples: {examples}
Use the following relation types, don't use other relation that is not defined below:
# RELATION TYPES:
{rel_types}

For the following input, extract entities and relationships as described in examples.
{format_instructions}
Input text: {input}
Extract meaningful string inside "()" if it is date and create relations using it only.If there no dates inside "()" give output as dictionay with keys as
["head","head_type","relation","tail","tail_type"] and their values as [None,None,None,None,None].

Special Note:DO NOT TAKE ANY OTHER DATE EXCEPT THE DATE INSIDE PARANTHESIS.
              
"""
,input_variables=["input"],partial_variables={"examples":doc_2,"rel_types":["CrimeCommited"],
        "format_instructions": parser.get_format_instructions()})


human_message_prompt=HumanMessagePromptTemplate(prompt=human_prompt)


# "Date" and "Crime"  with relation as "Convicted".Don't provide any other entity and relationship

In [165]:
system_message=SystemMessage(content=system_prompt)
chat_prompt=ChatPromptTemplate.from_messages([system_message,human_message_prompt])

In [166]:
chain = LLMChain(llm=llm, prompt=chat_prompt)

In [167]:
# texts=df.loc[0:150,"V_BIOGRAPHY_repo"].to_list()

In [168]:

llm = ChatOllama(model = "llama3", temperature=0,format="json")
chain = LLMChain(llm=llm, prompt=chat_prompt)
l_d=[]
bio_d=[]
ind_d=[]
index_d=0
time_d=[]
for i in texts:
    print(index_d)
    
    if i=="":
        l_d.append({})
        time_d.append(0)
        bio_d.append(i)
        ind_d.append(index_d)
        index_d=index_d+1
    else:
       start_d=time.time()
       output=chain.run(i)
       end_d=time.time()
       time_d.append(end_d-start_d)
       l_d.append(output)
       bio_d.append(i)
       ind_d.append(index_d)
       index_d=index_d+1

        

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38


In [169]:
# df_res=pd.DataFrame({})
# df_res["Biography"]=bio_d
df_res["groups_2"]=l_d
# df_res["index"]=ind_d


In [170]:
df_res["time_2"]=time_d

In [171]:
df_res["Biography"]=df["V_BIOGRAPHY_x"]

In [174]:
# df_res.to_csv("date_of_crime.csv")
df_res.to_csv("final_res_demo_2.csv")

In [172]:
pd.set_option("display.max_colwidth",None)

In [173]:
df_res

,Biography,groups,index,time,groups_2,time_2
0,"[BIOGRAPHY] Militant {Operations} of the Lashkar I Jhangvi. [IDENTIFICATION] To be determined. [FUNDING] To be determined. [REPORTS] Nov 2017 - detained by Pakistan Rangers Sindh in Mominabad for involvement in several cases of terrorism and attack on polio workers. Apr 2022 - no further information reported.[NONCONVICTION TERROR CATEGORY NOTICE] This category includes information about individuals who are reportedly being investigated for, or have been arrested or charged on suspicion of, involvement in terrorism or terror related activities. The category also includes individuals or entities identified on national or internationally recognised banning, warning or wanted lists as allegedly connected to terrorism or individuals who are reportedly connected to an organisation included on any national or international terrorism list. Inclusion in this category does not mean that an individual or entity is a terrorist or terrorist organisation or that they are involved in or connected to terrorism or terror related activity. Individuals included in this category have not been convicted of any terror related activity; you should review the content carefully and in accordance with our terms and conditions, further enquiries should be made of the report subject to investigate the outcome of any alleged investigation, arrest, charges or any reported connection to any terror related activity and whether such allegations are denied.","{\n""head"": ""Nov 2017"", \n""head_type"": ""Date"", \n""relation"": ""CrimeCommitted"", \n""tail"": ""terrorism and attack on polio workers"", \n""tail_type"": ""Crime""\n}\n\n",0,15.098829,"{\n ""output"": [\n {\n ""head"": ""Nov 2017"",\n ""head_type"": ""Date"",\n ""relation"": ""CrimeCommitted"",\n ""tail"": ""terrorism and attack on polio workers"",\n ""tail_type"": ""Crime""\n }\n ]\n}",6.351290
1,[BIOGRAPHY] To be determined. [IDENTIFICATION] Myo Htike (PEP) (spouse). Win Naing (father-in-law). Than Than Win (mother-in-law). Phoo Thitsar Wai (daughter). Eaint Eaint Phoo (daughter). [REPORTS] To be determined.,{},1,0.000000,{},0.000000
2,"[BIOGRAPHY] No known affiliation to any terrorist organisation. [IDENTIFICATION] Abdul Wahab (father). [FUNDING] To be determined. [REPORTS] Dec 2018 - reportedly detained. Sentenced to death for killing law enforcement agents and destruction of government educational institutions. Apr 2022 - no further information reported.[CRIME - TERROR CATEGORY NOTICE] This category includes information about individuals who have reportedly been convicted of a terror related crime. Inclusion in the category does not mean that an individual or entity is a terrorist or terrorist organisation. Individuals may appeal their conviction; you should review the content carefully and in accordance with our terms and conditions, further enquiries should be made of the report subject to investigate the conviction or the outcome of any appeal and whether the allegations of wrongdoing are denied.","{""head"": ""December 2018"", ""head_type"": ""Date"", ""relation"": ""Detained"", ""tail"": ""reportedly detained"", ""tail_type"": ""Event""}\n\n\n",2,5.301877,"{\n ""output"": [\n {\n ""head"": ""Dec 2018"",\n ""head_type"": ""Date"",\n ""relation"": ""CrimeCommitted"",\n ""tail"": ""killing law enforcement agents and destruction of government educational institutions"",\n ""tail_type"": ""Crime""\n }\n ]\n}",6.081783
3,"[BIOGRAPHY] No known affiliation to any terrorist organisation. [IDENTIFICATION] Abdul Wahab (father). [FUNDING] To be determined. [REPORTS] Dec 2018 - reportedly detained. Sentenced to death for killing law enforcement agents and destruction of government educational institutions. Apr 2022 - no further information reported.[CRIME - TERROR CATEGORY NOTICE] This category includes information about individuals who have reportedly been convicted of a terror related crime. Inclusion in the category does not mean that an individual o

## The end